In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
path_to_data_folder = 'C:/Users/emred/OneDrive/Masaüstü/Docs/3rd Year/Spring/CENG404 - Special Topics in CENG/Repository/data'

In [3]:
ground_truth_df = pd.read_csv(path_to_data_folder + '/mimic-cxr-2.1.0-test-set-labeled.csv')
ground_truth_df = ground_truth_df.drop('No Finding', axis = 1)
ground_truth_df = ground_truth_df.rename(columns = {'Airspace Opacity' : 'Lung Opacity'})
ground_truth_df = ground_truth_df.fillna(0)

results_df = pd.read_json(path_to_data_folder + '/ai_generated_reports_under_dicom_folder_output.json')
labels_df = pd.json_normalize(results_df['labels'])
results_df = pd.concat([results_df.drop(columns=['labels']), labels_df], axis = 1)
results_df = results_df.replace({
        'Yes' : 1,
        'No' : 0,
        'Maybe' : -1,
        'Undefined' : np.nan
    }
)
results_df['patient_id'] = results_df["patient_id"].str.replace("p", "").astype(int)
results_df['report_name'] = results_df['report_name'].str.replace('s', '').str.replace('.txt', '').astype(int)
results_df = results_df.rename(columns = {
    'patient_id' : 'subject_id',
    'report_name' : 'study_id'
})
results_df = results_df.drop('subject_id', axis = 1)
results_df = results_df.fillna(0)

reordered_columns = [col for col in ground_truth_df.columns]
results_df = results_df[reordered_columns]


results_ids = {id for id in results_df['study_id']}
ground_truth_ids = {id for id in ground_truth_df['study_id']}

missing_ids_in_ground_truth = {id for id in ground_truth_ids if id not in results_ids}

ground_truth_df = ground_truth_df[~ground_truth_df['study_id'].isin(missing_ids_in_ground_truth)]

results_df = results_df.sort_values(by = 'study_id')
ground_truth_df = ground_truth_df.sort_values(by = 'study_id')

C:\Users\emred\AppData\Local\Temp\ipykernel_25516\3540890675.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  results_df = results_df.replace({


In [4]:
results_df

,study_id,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
33,50008596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0
282,50020371,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
540,50022945,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
149,50031776,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
475,50037292,0.0,0.0,0.0,1.0,-1.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,59947192,0.0,0.0,-1.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
50,59956784,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
587,59968351,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
201,59983953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [5]:
ground_truth_df

,study_id,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
671,50008596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
2,50020371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
470,50022945,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
387,50031776,0.0,0.0,0.0,1.0,0.0,1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0
412,50037292,0.0,0.0,0.0,0.0,-1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,59947192,0.0,0.0,1.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
552,59956784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
11,59968351,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
499,59983953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [6]:
ground_truth_df.reset_index().drop('index', axis=1)

,study_id,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,50008596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
1,50020371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,50022945,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,50031776,0.0,0.0,0.0,1.0,0.0,1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0
4,50037292,0.0,0.0,0.0,0.0,-1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,59947192,0.0,0.0,1.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
681,59956784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
682,59968351,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
683,59983953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [7]:
def build_eval_metrics_df(results_df, ground_truth_df):
        # Merge the results and ground truth dataframes
    merged = pd.merge(results_df, ground_truth_df, on=['study_id'], suffixes=('_results', '_ground_truth'))
    
    # Initialize a list to store results
    results = []
    
    # Columns to compare (excluding 'study_id')
    columns_to_compare = [col for col in results_df.columns if col not in ['study_id']]
    
    # Iterate through each column to calculate metrics
    for col in columns_to_compare:
        # Get the predictions and ground truth for the current column
        y_pred = merged[f"{col}_results"]
        y_true = merged[f"{col}_ground_truth"]
    
        # Filter out rows where either y_pred or y_true is NaN (Undefined)
        valid_indices = ~y_pred.isna() & ~y_true.isna()
        y_pred = y_pred[valid_indices]
        y_true = y_true[valid_indices]
    
        # Total valid rows
        total_valid = len(y_true)
    
        # Calculate matches for accuracy
        matches = (y_pred == y_true).sum()
        accuracy = (matches / total_valid) * 100 if total_valid > 0 else 0
    
        # Calculate True Positives (TP), False Positives (FP), and False Negatives (FN) for each class
        tp = ((y_pred == 1) & (y_true == 1)).sum()  # True Positives for "Yes"
        fp = ((y_pred == 1) & (y_true != 1)).sum()  # False Positives for "Yes"
        fn = ((y_pred != 1) & (y_true == 1)).sum()  # False Negatives for "Yes"
    
        # Precision, Recall, and F1-Score for "Yes" class
        precision = (tp / (tp + fp) * 100) if (tp + fp) > 0 else 0
        recall = (tp / (tp + fn) * 100) if (tp + fn) > 0 else 0
        f1_score = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0
    
        # Append results for the current column
        results.append({
            'Category': col,
            'Total Valid': total_valid,
            'Matches': matches,
            'Accuracy': f"{accuracy:.2f}%",
            'Precision': f"{precision:.2f}%",
            'Recall': f"{recall:.2f}%",
            'F1-Score': f"{f1_score:.2f}%"
        })
    
    # Create a DataFrame to display the results
    accuracy_df = pd.DataFrame(results)
    
    # Display the resulting DataFrame
    return accuracy_df

#####################################################################################################################

# This function filters the -1's (Maybes)
def build_filtered_eval_metrics_df(results_df, ground_truth_df):
    merged = pd.merge(results_df, ground_truth_df, on=['study_id'], suffixes=('_results', '_ground_truth'))
    # Iterate through each category to calculate metrics
    results = []
    categories = [col.replace("_results", "") for col in merged.columns if "_results" in col]
    
    for category in categories:
        # Filter valid rows for the current category
        y_pred = merged[f"{category}_results"]
        y_true = merged[f"{category}_ground_truth"]
        valid_indices = (y_pred.isin([0, 1])) & (y_true.isin([0, 1]))
        
        y_pred = y_pred[valid_indices]
        y_true = y_true[valid_indices]
        
        # Total valid rows
        total_valid = len(y_true)
        
        # Calculate matches for accuracy
        matches = (y_pred == y_true).sum()
        accuracy = (matches / total_valid) * 100 if total_valid > 0 else 0
        
        # Calculate True Positives (TP), False Positives (FP), and False Negatives (FN)
        tp = ((y_pred == 1) & (y_true == 1)).sum()
        fp = ((y_pred == 1) & (y_true == 0)).sum()
        fn = ((y_pred == 0) & (y_true == 1)).sum()
        
        # Precision, Recall, and F1-Score
        precision = (tp / (tp + fp) * 100) if (tp + fp) > 0 else 0
        recall = (tp / (tp + fn) * 100) if (tp + fn) > 0 else 0
        f1_score = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0
        
        # Append results for the current category
        results.append({
            'Category': category,
            'Total Valid': total_valid,
            'Matches': matches,
            'Accuracy': f"{accuracy:.2f}%",
            'Precision': f"{precision:.2f}%",
            'Recall': f"{recall:.2f}%",
            'F1-Score': f"{f1_score:.2f}%"
        })
    
    # Create a DataFrame to display the results
    filtered_metrics_df = pd.DataFrame(results)
    
    # Display the resulting DataFrame
    return filtered_metrics_df

In [8]:
build_eval_metrics_df(results_df, ground_truth_df)

,Category,Total Valid,Matches,Accuracy,Precision,Recall,F1-Score
0,Enlarged Cardiomediastinum,685,590,86.13%,43.48%,21.74%,28.99%
1,Cardiomegaly,685,536,78.25%,62.89%,80.79%,70.72%
2,Lung Lesion,685,642,93.72%,73.91%,64.15%,68.69%
3,Lung Opacity,685,526,76.79%,53.74%,72.19%,61.62%
4,Edema,685,578,84.38%,71.03%,76.30%,73.57%
5,Consolidation,685,633,92.41%,70.37%,67.86%,69.09%
6,Pneumonia,685,610,89.05%,76.92%,54.55%,63.83%
7,Atelectasis,685,551,80.44%,70.45%,78.68%,74.34%
8,Pneumothorax,685,658,96.06%,82.14%,60.53%,69.70%
9,Pleural Effusion,685,575,83.94%,83.52%,83.52%,83.52%


In [9]:
build_filtered_eval_metrics_df(results_df, ground_truth_df)

,Category,Total Valid,Matches,Accuracy,Precision,Recall,F1-Score
0,Enlarged Cardiomediastinum,623,589,94.54%,52.63%,28.57%,37.04%
1,Cardiomegaly,606,521,85.97%,64.55%,87.14%,74.16%
2,Lung Lesion,666,639,95.95%,77.27%,66.67%,71.58%
3,Lung Opacity,671,526,78.39%,53.74%,75.31%,62.72%
4,Edema,625,565,90.40%,76.30%,78.63%,77.44%
5,Consolidation,641,614,95.79%,74.51%,73.08%,73.79%
6,Pneumonia,568,553,97.36%,88.24%,73.17%,80.00%
7,Atelectasis,613,532,86.79%,74.16%,85.16%,79.28%
8,Pneumothorax,671,655,97.62%,92.00%,62.16%,74.19%
9,Pleural Effusion,630,560,88.89%,85.07%,88.37%,86.69%
